___
<img style="float: right; margin: 0px 0px 15px 15px;" src="https://miro.medium.com/max/564/0*ToYXqRes95eMvIKV.png" width="250px" height="80px" />


# <font color= #8A0829> Laboratorio de Modelado de Datos </font>
#### <font color= #2E9AFE> `Martes y Viernes (Videoconferencia) de 13:00 - 15:00 hrs`</font>
- <Strong> Sara Eugenia Rodríguez </Strong>
- <Strong> Año </Strong>: 2021
- <Strong> Email: </Strong>  <font color="blue"> `cd682324@iteso.mx` </font>
___

<p style="text-align:right;"> Imagen recuperada de: https://miro.medium.com/max/564/0*ToYXqRes95eMvIKV.png</p>

### <font color= #2E9AFE> Tema: Modelos basados en Árboles - Clasificación</font>

Un árbol de decisión es un conjunto de sentencias de la forma: si... entonces...

En un árbol de decisión de clasificación, el árbol busca crear preguntas secuenciales de forma que parta los datos en grupos más pequeños. 

Una vez que las particiones están hechas, una decisión predictiva es hecha a través del nodo final (basado en frecuencias).

**Estructura del árbol**

Toma 3 cosas en cuenta:
- Las variables predictoras (X) que se van a usar y el punto de partición del dataset.
- La profundidad/complejidad del árbol
- La ecuación de predicción en los últimos nodos/hojas del árbol

**Hiperparámetros a ajustar**
- Profundidad del árbol (max_depth)
- Número mínimo de observaciones en cada split(min_samples_split)

**Gini Impurity**
Esta medida nos dice cuál es la probabilidad de clasificar erróneamente una observación. 

*Nota*: mientras más pequeño el valor de Gini, mejor el split. En otras palabras menor la probabilidad de clasificar erróneamente. 

$$Gini = p_{1}(1-p_{1})+p_{2}(1-p_{2})=2(p_{1})(p_{2})$$

Donde $p_{1}$,$p_{2}$ son las probabilidades de la clase 1 y 2 respectivamente. 

Esto no está totalmente completo. La ecuación anterior da el gini impurity para cada sub-partición, pero el objetivo es conocer el gini impurity para toda la partición (porque los datos se dividen a la izquierda y a la derecha). Por lo tanto, necesitamos ponderarlos acordemente:

$$Gini_{ponderado} = p_{I}(2(p_{I1})(p_{I2}))+p_{D}(2(p_{D1})(p_{D2}))$$
 
Donde I=Izquierda, D=Derecha

**Desventajas**

- Inestabilidad del modelo: Debido a que las particiones se basan en un conjunto de datos, si se generan cambios en el conjunto de datos, esto genera cambios importantes en la estructura del árbol y especialmente en su interpretabilidad.

- Rendimiento predictivo subóptimo. Nuevamente, debido a que las particiones se basan en un conjunto de datos específico, el modelo generalmente no converge con el modelo óptimo global.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification
import pandas as pd
from sklearn.metrics import (accuracy_score,precision_score,recall_score)
from sklearn.datasets import make_gaussian_quantiles
from sklearn.datasets import make_blobs

In [ ]:
#Generar los datos
plt.title("Gausiana dividida en 3 clases", fontsize='small')
X1, Y1 = make_gaussian_quantiles(n_samples = 500,n_features=2, n_classes=3)
plt.scatter(X1[:, 0], X1[:, 1], marker='o', c=Y1,
            s=25, edgecolor='k')

plt.show()

In [ ]:
#Dividir datos en train y test
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X1, Y1, test_size=0.20,
                                                    random_state=0,
                                                    shuffle=True)

In [ ]:
X1

In [ ]:
Y1

In [ ]:
max_depths = range(1, 20)

#para el train
training_acc = []
for max_depth in max_depths:
    model_1 = DecisionTreeClassifier(max_depth=max_depth)
    model_1.fit(X1, Y1)
    training_acc.append(accuracy_score(Y1, model_1.predict(X1)))
    
#para el test    
testing_acc = []
for max_depth in max_depths:
    model_2 = DecisionTreeClassifier(max_depth=max_depth)
    model_2.fit(X_train, y_train)
    testing_acc.append(accuracy_score(y_test, model_2.predict(X_test)))


In [ ]:
plt.plot(max_depths, training_acc, color='blue', label='Training Accuracy')
plt.plot(max_depths, testing_acc, color='green', label='Testing Accuracy')
plt.xlabel('Tree depth')
plt.ylabel('Accuracy')
plt.title('Hyperparameter Tuning', pad=15, size=15)
plt.legend()

In [ ]:
#Usando cross validation y grid search
from sklearn.model_selection import GridSearchCV

model = DecisionTreeClassifier()

gs = GridSearchCV(model,
                  param_grid = {'max_depth': range(1, 11),
                                'min_samples_split': range(10, 60, 10)},
                  cv=5,
                  scoring='accuracy')

gs.fit(X_train, y_train)

print(gs.best_params_)

In [ ]:
#crear modelo usando parámetros óptimos
new_model = DecisionTreeClassifier(max_depth=7,
                                  min_samples_split=10)
new_model.fit(X_train, y_train)

In [ ]:
from sklearn.metrics import r2_score
yhat = new_model.predict(X_test)
accuracy = accuracy_score(y_test,yhat)
print('Accuracy:', accuracy)

In [ ]:
new_model.feature_importances_

In [ ]:
from sklearn import tree
import graphviz
dot_data = tree.export_graphviz(new_model, out_file=None)
graph = graphviz.Source(dot_data)
graph.render("trees_gen/tree_multipredict")

**Ventajas de los árboles de decisión**

- No requiere escalamiento de variables
- Puede  ser usado para datos no lineales
- Fácil de visualizar
- Fácil de interpretar

**Desventajas de los árboles de decisión**

- Es computancionalmente complejo, especialmente al usar cross-validation para ajustar los hiperparámetros
- Un cambio pequeño en los datos puede causar grandes cambios en la estructura del árbol